In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install SimpleITK
!pip install monai
!pip install --upgrade monai
# !pip install torch-lr-finder

In [ ]:
import pickle
import SimpleITK as sitk
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import ConcatDataset

import monai
from monai.networks.nets import DenseNet121
from monai.metrics import ROCAUCMetric
from monai.transforms import Compose, AddChannel, ScaleIntensity, ToTensor
from monai.transforms import Spacing, create_grid
from monai.transforms import RandRotate90, RandAdjustContrast, RandZoom, RandFlip, RandGaussianNoise, RandGibbsNoise, RandKSpaceSpikeNoise

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# from torch_lr_finder import LRFinder

# Deep Learning

In [ ]:
############################ DATASET ####################################

class MRI_Dataset(Dataset):
  def __init__(self, task, fold, modality, transforms):
    self.task = task
    self.fold = fold
    self.modality = modality
    self.transform=transforms
    self.lib, self.target = pickle.load(open("/content/drive/My Drive/CTM/multimodal_glioma_data_multi_level.pickle","rb"))[fold][task]
    self.data_path = pickle.load(open(f'/content/drive/My Drive/CTM/tabular_features_10_tiles/t1ce_flair/global/{task}_{fold}.pkl',"rb"))[0]["path"]

  def __len__(self):
    return len(self.target)

  def targetTransform(self, target):

    if(target['idh1'] == 0 and target['ioh1p19q'] == 0):
      return 0
    if(target['idh1'] == 1 and target['ioh1p19q'] == 0):
      return 1
    if(target['idh1'] == 1 and target['ioh1p19q'] == 1):
      return 2

  def __getitem__(self, patientID):
    path = self.lib[patientID][self.modality]

    map_dict = {'\\': '/'}
    path = ''.join(idx if idx not in map_dict else map_dict[idx] for idx in path)
    image_url = '/content/drive/My Drive/CTM/{}'.format(path)

    pixels_array = np.load(image_url, allow_pickle=True)
    pixels_array= self.transform(pixels_array)

    return [pixels_array, self.data_path[patientID]], self.targetTransform(self.target[patientID])

In [ ]:
######################## TRANSFORMATIONS ########################

train_transforms= Compose([ScaleIntensity(), RandRotate90(), RandAdjustContrast()])
val_transforms= Compose([ScaleIntensity()])

In [ ]:
ds_test = MRI_Dataset('test', 4, 't1', val_transforms)

# # t1ce + flair
ds_train1 = MRI_Dataset('train', 4, 't1', train_transforms)
ds_train2 = MRI_Dataset('train', 4, 't2', train_transforms)

concat_dataset_train = ConcatDataset([ds_train1, ds_train2])

train_dataloader = DataLoader(ds_train1, batch_size=32, shuffle=True, num_workers=2)
test_dataloader = DataLoader(ds_test, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
class LinearDenseNet121Head(nn.Module):
    def __init__(self, spatial_dims, in_channels, out_channels, dropout_prob):
        super(LinearDenseNet121Head, self).__init__()

        self.model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=1024, pretrained=False, dropout_prob=dropout_prob).to(device)

        #  Linear layer to replace AdaptiveAvgPool3d and ConvolutionalLayer
        self.linear = nn.Sequential(#nn.Linear(512+5120,128),
                                    nn.Linear(1024,128),
                                    nn.ReLU(),
                                    nn.Dropout(dropout_prob),
                                    nn.Linear(128,32),
                                    nn.ReLU(),
                                    nn.Dropout(dropout_prob),
                                    nn.Linear(32,out_channels))


    def forward(self, x, data):
        output_conv = self.model(x)
        # pathData = data
        # output = torch.cat((output, pathData), dim=1)
        output = self.linear(output_conv)
        return output_conv, output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LinearDenseNet121Head(spatial_dims=3, in_channels=1, out_channels=3, dropout_prob=0.2).to(device)
# model.head = LinearDenseNet121Head(spatial_dims=3, in_channels=128, out_channels=3, dropout_prob=1e-3)
loss_function = torch.nn.CrossEntropyLoss()
lr=1e-4
weight_decay=1e-5
optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=weight_decay)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience = 5, threshold=0.01)
max_epochs = 100

In [ ]:
#################### MODEL TRAIN

best_metric = -1
best_metric_epoch = -1

best_val_accuracy = 0.0
best_model_weights = None

epoch_loss_values = []

for epoch in range(max_epochs):

    print("-" * 10)
    print(f"EPOCH {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    batch_iter = 0

    print("Learning Rate= ", optimizer.param_groups[0]['lr'])

    for data, labels in train_dataloader:

      if torch.cuda.is_available():
        data[0], labels = data[0].cuda(), labels.cuda()
        data[1] = data[1].cuda()

      batch_iter += 1
      optimizer.zero_grad()
      _, outputs = model(torch.unsqueeze(data[0],1), data[1])
      loss = loss_function(outputs, labels)
      loss.backward()
      optimizer.step()
      epoch_loss += loss.item()
      # epoch_len = len(ds_train) // train_dataloader.batch_size

    epoch_loss /= batch_iter
    epoch_loss_values.append(epoch_loss)
    print("")
    print(f"epoch {epoch + 1} average loss: {epoch_loss}")

    scheduler.step(epoch_loss)

    # Validation

    model.eval()
    val_loss = 0
    total = 0
    correct = 0
    predicted_labels = []
    true_labels = []

    predicted_labels.clear()
    true_labels.clear()

    with torch.no_grad():

      for data, labels in test_dataloader:
          if torch.cuda.is_available():
            data[0], labels = data[0].cuda(), labels.cuda()
            data[1] = data[1].cuda()

          _, outputs = model(torch.unsqueeze(data[0],1), data[1])
          loss = loss_function(outputs, labels)
          val_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)

          predicted_labels.extend(predicted.cpu().numpy())
          true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(true_labels, predicted_labels) * 100
    val_loss /= len(test_dataloader)

    print("")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}%")


    # weight freeze

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_model_weights = model.state_dict()
        torch.save(best_model_weights, "best_model_weights.pth")

print("")
print("-" * 10)
print(f"BEST MODEL ACCURACY ->: {best_val_accuracy:.4f}%")

----------
EPOCH 1/100
Learning Rate=  0.0001

epoch 1 average loss: 1.0501619338989259

Validation Loss: 1.0476
Validation Accuracy: 56.7568%
----------
EPOCH 2/100
Learning Rate=  0.0001

epoch 2 average loss: 1.0202269792556762

Validation Loss: 1.0049
Validation Accuracy: 56.7568%
----------
EPOCH 3/100
Learning Rate=  0.0001

epoch 3 average loss: 0.9863813519477844

Validation Loss: 0.9705
Validation Accuracy: 56.7568%
----------
EPOCH 4/100
Learning Rate=  0.0001

epoch 4 average loss: 0.9623837828636169

Validation Loss: 0.9483
Validation Accuracy: 56.7568%
----------
EPOCH 5/100
Learning Rate=  0.0001

epoch 5 average loss: 0.9459714889526367

Validation Loss: 0.9380
Validation Accuracy: 56.7568%
----------
EPOCH 6/100
Learning Rate=  0.0001

epoch 6 average loss: 0.9210052371025086

Validation Loss: 0.9350
Validation Accuracy: 56.7568%
----------
EPOCH 7/100
Learning Rate=  0.0001

epoch 7 average loss: 0.9515706896781921

Validation Loss: 0.9409
Validation Accuracy: 56.7568%

# Machine Learning

In [ ]:
!pip install catboost

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, mean_absolute_error, auc, roc_curve, f1_score, make_scorer, roc_auc_score


In [ ]:
def computeMetrics(y_test, y_pred, y_pred_prob, test_labels) :
# Compute metrics - Accuracy, Mean Absolute Error, F1 Score, AUC
    accuracy = accuracy_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    auc_score = roc_auc_score(test_labels, y_pred_prob, average='weighted', multi_class='ovr')

    print('Accuracy:', round(accuracy,6)*100)
    print('Mean Absolute Error:', round(mae,3))
    print('F1 Score:',  round(f1,6)*100)
    print('AUC:', round(auc_score,3))

    print("Confusion Matrix : \n", confusion_matrix(y_test, y_pred))

In [ ]:
fold = 4
multi = False

ds_test = MRI_Dataset('test', fold, 't1', val_transforms)

ds_train = MRI_Dataset('train', fold, 't1', train_transforms)
ds_train2 = MRI_Dataset('train', fold, 't2', train_transforms)
concat_dataset_train = ConcatDataset([ds_train, ds_train2])


if(multi):
  ds_train = concat_dataset_train

train_dataloader = DataLoader(ds_train, batch_size=1, shuffle=True, num_workers=2)
test_dataloader = DataLoader(ds_test, batch_size=1, shuffle=False, num_workers=2)

ml_feat_mri_train = []
ml_feat_path_train = []
ml_labels_train = []

model.eval()
with torch.no_grad():
      for data, labels in train_dataloader:

        if torch.cuda.is_available():
          data[0], labels = data[0].cuda(), labels.cuda()
          data[1] = data[1].cuda()

        ml_feat_path_train.append(torch.squeeze(data[1].cpu()).numpy())

        feature_vector, output = model(torch.unsqueeze(data[0],1), data[1])
        ml_feat_mri_train.append(torch.squeeze(feature_vector.cpu()).numpy())
        ml_labels_train.append(labels.cpu().numpy())

ml_feat_mri_test = []
ml_feat_path_test = []
ml_labels_test = []
with torch.no_grad():
      for data, labels in test_dataloader:
        # gc.collect()
        # torch.cuda.empty_cache()

        if torch.cuda.is_available():
          data[0], labels = data[0].cuda(), labels.cuda()
          data[1] = data[1].cuda()

        ml_feat_path_test.append(torch.squeeze(data[1].cpu()).numpy())

        feature_vector, output = model(torch.unsqueeze(data[0],1), data[1])
        ml_feat_mri_test.append(torch.squeeze(feature_vector.cpu()).numpy())
        ml_labels_test.append(labels.cpu().numpy())

In [ ]:
def getMetricsForClassfier(classifier):

  classifier.fit(ml_feat_mri_train, np.ravel(ml_labels_train)) # using np.ravel to fix data conversion warning

  y_pred = classifier.predict(ml_feat_mri_test)

  y_pred_prob = classifier.predict_proba(ml_feat_mri_test)
  computeMetrics(ml_labels_test, y_pred, y_pred_prob, ml_labels_test)

##LR

In [ ]:
# params = {'C': 1.0, 'max_iter': 800, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'saga'}

# lr_clf = LogisticRegression(**params)


# getMetricsForClassfier(lr_clf)

##KNN

In [ ]:
# knn_clf=KNeighborsClassifier(n_neighbors = 8)

# getMetricsForClassfier(knn_clf, 0, False)

##RF

In [ ]:
# params = {'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_split': 8, 'n_estimators': 200}

# rf_clf = RandomForestClassifier(**params)

# getMetricsForClassfier(rf_clf, 0, False)

##SVC

In [ ]:
# params = {'C': 1, 'gamma': 0.1, 'kernel': 'linear', 'probability': True}


# svc_clf = SVC(**params)

# getMetricsForClassfier(svc_clf, 0, False)

##XGB

In [ ]:
# xgb_clf=XGBClassifier(colsample_bytree= 0.6, gamma= 3, max_depth= 5, min_child_weight= 1, predictor= 'gpu_predictor', subsample= 0.8)

# getMetricsForClassfier(xgb_clf, 0, False)

#CAT

In [ ]:
cat_clf = CatBoostClassifier(bagging_temperature= 10,boosting_type= 'Plain', depth= 4,
                          iterations= 500, l2_leaf_reg= 1, leaf_estimation_iterations= 1,
                          learning_rate= 0.05, silent= False, task_type= 'GPU',loss_function='MultiClass' )

getMetricsForClassfier(cat_clf)

0:	learn: 1.0416803	total: 33.8ms	remaining: 16.9s
1:	learn: 0.9863330	total: 66.2ms	remaining: 16.5s
2:	learn: 0.9341041	total: 97.3ms	remaining: 16.1s
3:	learn: 0.8843665	total: 127ms	remaining: 15.7s
4:	learn: 0.8403735	total: 158ms	remaining: 15.6s
5:	learn: 0.8050842	total: 189ms	remaining: 15.6s
6:	learn: 0.7691502	total: 213ms	remaining: 15s
7:	learn: 0.7375887	total: 271ms	remaining: 16.7s
8:	learn: 0.7079518	total: 310ms	remaining: 16.9s
9:	learn: 0.6812886	total: 333ms	remaining: 16.3s
10:	learn: 0.6582684	total: 360ms	remaining: 16s
11:	learn: 0.6347606	total: 393ms	remaining: 16s
12:	learn: 0.6134054	total: 426ms	remaining: 16s
13:	learn: 0.5951540	total: 456ms	remaining: 15.8s
14:	learn: 0.5769996	total: 491ms	remaining: 15.9s
15:	learn: 0.5573888	total: 525ms	remaining: 15.9s
16:	learn: 0.5436023	total: 559ms	remaining: 15.9s
17:	learn: 0.5309160	total: 593ms	remaining: 15.9s
18:	learn: 0.5175703	total: 626ms	remaining: 15.9s
19:	learn: 0.5028592	total: 644ms	remaining: 1